---
execute:
  cache: false
  eval: true
  echo: true
  warning: false
jupyter: python3
title: Explainable AI with SpotPython and Pytorch
---

In [1]:
# from spotpython.utils.init import fun_control_init
# from spotpython.data.diabetes import Diabetes
# from spotpython.hyperdict.light_hyper_dict import LightHyperDict
# from spotpython.hyperparameters.values import (
#         get_default_hyperparameters_as_array, get_one_config_from_X)
# from spotpython.plot.xai import (get_activations, get_gradients, get_weights, plot_nn_values_hist, plot_nn_values_scatter, visualize_weights, visualize_gradients, visualize_activations, visualize_gradient_distributions, visualize_weights_distributions)

# data_set = Diabetes()
# fun_control = fun_control_init(
#     data_set=data_set,
#     _L_in=10, 
#     _L_out=1,
#     _torchmetric="mean_squared_error",
#     core_model_name="light.regression.NNLinearRegressor",
#     hyperdict=LightHyperDict,
#     )
# X = get_default_hyperparameters_as_array(fun_control)
# config = get_one_config_from_X(X, fun_control)
# _L_in = fun_control["_L_in"]
# _L_out = fun_control["_L_out"]
# _torchmetric = fun_control["_torchmetric"]
# model = fun_control["core_model"](**config, _L_in=_L_in, _L_out=_L_out, _torchmetric=_torchmetric)
# batch_size=config["batch_size"]

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from spotpython.data.diabetes import Diabetes
from spotpython.hyperdict.light_hyper_dict import LightHyperDict
from spotpython.fun.hyperlight import HyperLight
from spotpython.utils.init import (fun_control_init, surrogate_control_init, design_control_init)
from spotpython.utils.eda import gen_design_table
from spotpython.spot import spot
from spotpython.utils.file import get_experiment_filename
from spotpython.hyperparameters.values import set_hyperparameter
from math import inf

PREFIX="602"

data_set = Diabetes()

fun_control = fun_control_init(
    save_experiment=True,
    PREFIX=PREFIX,
    fun_evals=inf,
    max_time=60,
    data_set = data_set,
    core_model_name="light.regression.NNLinearRegressor",
    hyperdict=LightHyperDict,
    _L_in=10,
    _L_out=1)

fun = HyperLight().fun


set_hyperparameter(fun_control, "optimizer", [ "Adadelta", "Adam", "Adamax"])
set_hyperparameter(fun_control, "l1", [2,3])
set_hyperparameter(fun_control, "epochs", [10,12])
set_hyperparameter(fun_control, "batch_size", [4,11])
set_hyperparameter(fun_control, "dropout_prob", [0.0, 0.025])
set_hyperparameter(fun_control, "patience", [2,9])

design_control = design_control_init(init_size=20)

spot_tuner = spot.Spot(fun=fun,fun_control=fun_control, design_control=design_control)

Seed set to 123


Seed set to 123


module_name: light
submodule_name: regression
model_name: NNLinearRegressor


### Running the Hyperparameter Tuning or Loading the Existing Model


In [4]:
from spotpython.utils.file import get_experiment_filename, load_experiment
import os
overwrite = False
filename = get_experiment_filename(PREFIX)
if os.path.exists(filename) and not overwrite:
    (spot_tuner, fun_control, design_control,
    surrogate_control, optimizer_control) = load_experiment(filename)
else:
    print("File does not exist or overwrite is True. Starting new experiment.")
    res = spot_tuner.run()

Loaded experiment from spot_602_experiment.pickle


### Results from the Hyperparameter Tuning Experiment

* After the hyperparameter tuning is finished, the following information is available:
    * the `spot_tuner` object and the associated
    * `fun_control` dictionary


In [5]:
res = spot_tuner.print_results(print_screen=True)

min y: 2909.064208984375
l1: 3.0
epochs: 11.0
batch_size: 11.0
act_fn: 5.0
optimizer: 2.0
dropout_prob: 0.025
lr_mult: 10.0
patience: 7.0
batch_norm: 0.0
initialization: 2.0


#### Getting the Best Model, i.e, the Tuned Architecture

* The method `get_tuned_architecture` [[DOC]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotpython/hyperparameters/values/#spotpython.hyperparameters.values.get_tuned_architecture) returns the best model architecture found during the hyperparameter tuning.
* It returns the transformed values, i.e., `batch_size = 2^x` if the hyperparameter `batch_size` was transformed with the `transform_power_2_int` function.


In [6]:
from spotpython.hyperparameters.values import get_tuned_architecture
import pprint
config = get_tuned_architecture(spot_tuner, fun_control)
pprint.pprint(config)

{'act_fn': Swish(),
 'batch_norm': False,
 'batch_size': 2048,
 'dropout_prob': 0.025,
 'epochs': 2048,
 'initialization': 'kaiming_normal',
 'l1': 8,
 'lr_mult': 10.0,
 'optimizer': 'Adamax',
 'patience': 128}


* Note: `get_tuned_architecture` has the option `force_minX` which does not have any effect in this case.


In [7]:
from spotpython.hyperparameters.values import get_tuned_architecture
config = get_tuned_architecture(spot_tuner, fun_control, force_minX=True)
pprint.pprint(config)

{'act_fn': Swish(),
 'batch_norm': False,
 'batch_size': 2048,
 'dropout_prob': 0.025,
 'epochs': 2048,
 'initialization': 'kaiming_normal',
 'l1': 8,
 'lr_mult': 10.0,
 'optimizer': 'Adamax',
 'patience': 128}


### Training the Tuned Architecture on the Test Data

* Since we are interested in the explainability of the model, we will train the tuned architecture on the test data.
* `spotpythons`'s `test_model` function [[DOC]](https://sequential-parameter-optimization.github.io/spotPython/reference/spotpython/light/testmodel/) is used to train the model on the test data.
* Note: Until now, we do not use any information about the NN's weights and biases. Only the architecture, which is available as the `config`, is used.
* `spotpython` used the TensorBoard logger to save the training process in the `./runs` directory. Therefore, we have to enable the TensorBoard logger in the `fun_control` dictionary. To get a clean start, we also set the `TENSORBOARD_CLEAN` flag to `True`.


In [8]:
from spotpython.light.testmodel import test_model
from spotpython.light.loadmodel import load_light_from_checkpoint

fun_control.update({"TENSORBOARD_CLEAN": True})
fun_control.update({"tensorboard_log": True})
test_model(config, fun_control)

GPU available: True (mps), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs



  | Name   | Type       | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------
0 | layers | Sequential | 169    | train | [2048, 10] | [2048, 1]
-----------------------------------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


Restoring states from the checkpoint path at /Users/bartz/workspace/Hyperparameter-Tuning-Cookbook/runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TEST/last-v1.ckpt


Loaded model weights from the checkpoint at /Users/bartz/workspace/Hyperparameter-Tuning-Cookbook/runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TEST/last-v1.ckpt


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         hp_metric         │     3530.435791015625     │
│         val_loss          │     3530.435791015625     │
└───────────────────────────┴───────────────────────────┘

test_model result: {'val_loss': 3530.435791015625, 'hp_metric': 3530.435791015625}


(3530.435791015625, 3530.435791015625)

In [9]:
model = load_light_from_checkpoint(config, fun_control)

config: {'l1': 8, 'epochs': 2048, 'batch_size': 2048, 'act_fn': Swish(), 'optimizer': 'Adamax', 'dropout_prob': 0.025, 'lr_mult': 10.0, 'patience': 128, 'batch_norm': False, 'initialization': 'kaiming_normal'}
Loading model with 8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TEST from runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TEST/last.ckpt
Model: NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
    (12): Linear(in_features=2, out_features=2, bias=True)
    (13): Swish()
    (14): Dr

##### Details of the Training Process on the Test Data

* The `test_model` method initializes the model with the tuned architecture as follows:

```python
model = fun_control["core_model"](**config, _L_in=_L_in, _L_out=_L_out, _torchmetric=_torchmetric)
```

* Then, the Lightning Trainer is initialized with the `fun_control` dictionary and the model as follows:
    
    ```python
        trainer = L.Trainer(
        default_root_dir=os.path.join(fun_control["CHECKPOINT_PATH"], config_id),
        max_epochs=model.hparams.epochs,
        accelerator=fun_control["accelerator"],
        devices=fun_control["devices"],
        logger=TensorBoardLogger(
            save_dir=fun_control["TENSORBOARD_PATH"],
            version=config_id,
            default_hp_metric=True,
            log_graph=fun_control["log_graph"],
        ),
        callbacks=[
            EarlyStopping(monitor="val_loss", patience=config["patience"], mode="min", strict=False, verbose=False),
            ModelCheckpoint(
                dirpath=os.path.join(fun_control["CHECKPOINT_PATH"], config_id), save_last=True
            ), 
        ],
        enable_progress_bar=enable_progress_bar,
    )
    trainer.fit(model=model, datamodule=dm)    
    test_result = trainer.test(datamodule=dm, ckpt_path="last")
    ```

* As shown in the code above, the last checkpoint ist saved.
* `spotpython`'s method `load_light_from_checkpoint` is used to load the last checkpoint and to get the model's weights and biases. It requires the `fun_control` dictionary and the `config_id` as input to find the correct checkpoint.
* Now, the model is trained and the weights and biases are available.


In [10]:
pprint.pprint(model)

NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
    (12): Linear(in_features=2, out_features=2, bias=True)
    (13): Swish()
    (14): Dropout(p=0.025, inplace=False)
    (15): Linear(in_features=2, out_features=2, bias=True)
    (16): Swish()
    (17): Dropout(p=0.025, inplace=False)
    (18): Linear(in_features=2, out_features=1, bias=True)
  )
)


In [11]:
from torchviz import make_dot
import torch

x = torch.randn(1, 10).requires_grad_(True)
x = x.to("mps")
output = model(x)
dot = make_dot(output, params=dict(model.named_parameters()), show_attrs=True, show_saved=True)
dot.render("model_architecture", format="png")

'model_architecture.png'

![architecture](./model_architecture.png)

## XAI Methods

* `spotpython` provides methods to explain the model's predictions. The following neural network elements can be analyzed: 

### Weights

* Weights are the parameters of the neural network that are learned from the data during training. They connect neurons between layers and determine the strength and direction of the signal sent from one neuron to another. The network adjusts the weights during training to minimize the error between the predicted output and the actual output.
* Interpretation of the weights: A high weight value indicates a strong influence of the input neuron on the output. Positive weights suggest a positive correlation, whereas negative weights suggest an inverse relationship between neurons.

### Activations

* Activations are the outputs produced by neurons after applying an activation function to the weighted sum of inputs. The activation function (e.g., ReLU, sigmoid, tanh) adds non-linearity to the model, allowing it to learn more complex relationships.
* Interpretation of the activations: The value of activations indicates the intensity of the signal passed to the next layer. Certain activation patterns can highlight which features or parts of the data the network is focusing on.

### Gradients

* Gradients are the partial derivatives of the loss function with respect to different parameters (weights) of the network. During backpropagation, gradients are used to update the weights in the direction that reduces the loss by methods like gradient descent.
* Interpretation of the gradients: The magnitude of the gradient indicates how much a parameter should change to reduce the error. A large gradient implies a steeper slope and a bigger update, while a small gradient suggests that the parameter is near an optimal point. If gradients are too small (vanishing gradient problem), the network may learn slowly or stop learning. If they are too large (exploding gradient problem), the updates may be unstable.


In [12]:
from spotpython.plot.xai import (get_activations, get_gradients, get_weights, visualize_weights, visualize_gradients, visualize_activations, visualize_gradient_distributions, visualize_weights_distributions, visualize_activations_distributions)
batch_size = config["batch_size"]

### Getting the Weights


In [13]:
from spotpython.plot.xai import sort_layers
weights = get_weights(model)
sort_layers(weights)

{'Layer 0': array([-0.6739463 , -0.1643242 ,  0.94713277,  0.51443714, -0.84001887,
        -0.38128138, -0.32533002, -0.00294102,  2.4766104 , -0.13891017,
        -0.46667063, -0.43009487,  1.1950493 ,  0.87567604, -0.06863187,
        -0.3406367 , -0.47343305,  0.22405776,  1.6098766 , -0.23754378,
        -0.11924004, -0.48051828,  1.2585093 ,  0.69422036, -0.3035147 ,
        -0.08363269, -0.25447404, -0.46552902,  1.7778572 ,  0.40020478,
        -0.15642366, -0.07005566,  1.266909  ,  0.78304744, -0.46556482,
        -0.33197567, -0.49643585, -0.27119756,  1.3164715 ,  0.09442065,
        -0.33549333,  1.357861  , -0.92126226, -0.93145347,  0.61957085,
        -0.17734866,  1.3769729 , -0.7207191 , -1.1762294 , -0.5470856 ,
         0.30846247, -0.5025774 ,  0.8135039 ,  0.48299864, -0.4820734 ,
         0.25034988, -0.40744984,  0.3706972 ,  1.6866654 , -0.04715892,
        -0.25872642, -0.468488  ,  1.2969601 ,  1.5718246 , -0.7756649 ,
        -0.11315741, -0.8533421 ,  0.265

In [14]:
visualize_weights(model, absolute=True, cmap="GreenYellowRed", figsize=(6, 6))

80 values in Layer Layer 0.
1 padding values added.
81 values now in Layer Layer 0.


<Figure size 1800x1800 with 2 Axes>

32 values in Layer Layer 3.
4 padding values added.
36 values now in Layer Layer 3.


<Figure size 1800x1800 with 2 Axes>

16 values in Layer Layer 6.
16 values now in Layer Layer 6.


...

<Figure size 1800x1800 with 2 Axes>

4 values in Layer Layer 15.
4 values now in Layer Layer 15.


<Figure size 1800x1800 with 2 Axes>

2 values in Layer Layer 18.
2 padding values added.
4 values now in Layer Layer 18.


<Figure size 1800x1800 with 2 Axes>

In [15]:
visualize_weights_distributions(model, color=f"C{0}")

n:7


<Figure size 1620x3000 with 8 Axes>

### Getting the Activations


In [16]:
activations = get_activations(model, fun_control=fun_control, batch_size=batch_size, device = "mps")
print(activations)

{0: array([ 0.04758742,  0.02930558,  0.05538465,  0.00697784,  0.04627909,
       -0.04806035,  0.07788565,  0.0290162 ], dtype=float32), 3: array([ 0.07376944, -0.07319036,  0.17727649,  0.01832922], dtype=float32), 6: array([-0.51597834, -0.09564247,  0.25488186, -0.22002655], dtype=float32), 9: array([-0.49103516,  1.5572414 ], dtype=float32), 12: array([-5.058619  , -0.41804186], dtype=float32), 15: array([8.496492, 8.913045], dtype=float32), 18: array([152.00153], dtype=float32)}


In [17]:
visualize_activations(model, fun_control=fun_control, batch_size=batch_size, device = "mps", cmap="GreenYellowRed", absolute=True)

8 values in Layer 0.
1 padding values added.
9 values now in Layer 0.


<Figure size 1800x1800 with 2 Axes>

4 values in Layer 3.
4 values now in Layer 3.


<Figure size 1800x1800 with 2 Axes>

4 values in Layer 6.
4 values now in Layer 6.


...

<Figure size 1800x1800 with 2 Axes>

2 values in Layer 15.
2 padding values added.
4 values now in Layer 15.


<Figure size 1800x1800 with 2 Axes>

1 values in Layer 18.
1 values now in Layer 18.


<Figure size 1800x1800 with 2 Axes>

In [18]:
visualize_activations_distributions(net=model, fun_control=fun_control, batch_size=batch_size, device="mps", color="C0", columns=2)

n:7


<Figure size 1620x3000 with 8 Axes>

### Getting the Gradients


In [19]:
gradients = get_gradients(model, fun_control=fun_control, batch_size=batch_size, device = "mps")

In [20]:
visualize_gradients(model, fun_control, batch_size, absolute=True, cmap="GreenYellowRed", figsize=(6, 6), device="mps")

80 values in Layer layers.0.weight.
1 padding values added.
81 values now in Layer layers.0.weight.


<Figure size 1800x1800 with 2 Axes>

32 values in Layer layers.3.weight.
4 padding values added.
36 values now in Layer layers.3.weight.


<Figure size 1800x1800 with 2 Axes>

16 values in Layer layers.6.weight.
16 values now in Layer layers.6.weight.


...

<Figure size 1800x1800 with 2 Axes>

4 values in Layer layers.15.weight.
4 values now in Layer layers.15.weight.


<Figure size 1800x1800 with 2 Axes>

2 values in Layer layers.18.weight.
2 padding values added.
4 values now in Layer layers.18.weight.


<Figure size 1800x1800 with 2 Axes>

In [21]:
visualize_gradient_distributions(model, fun_control, batch_size=batch_size, color=f"C{0}", device="mps")

n:7


<Figure size 1620x3000 with 8 Axes>

## Feature Attributions

### Integrated Gradients


In [22]:
from spotpython.plot.xai import get_attributions, plot_attributions
df_att = get_attributions(spot_tuner, fun_control, attr_method="IntegratedGradients", n_rel=10)
plot_attributions(df_att, attr_method="IntegratedGradients")

GPU available: True (mps), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs



  | Name   | Type       | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------
0 | layers | Sequential | 169    | train | [2048, 10] | [2048, 1]
-----------------------------------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


train_model result: {'val_loss': 2918.2333984375, 'hp_metric': 2918.2333984375}
config: {'l1': 8, 'epochs': 2048, 'batch_size': 2048, 'act_fn': Swish(), 'optimizer': 'Adamax', 'dropout_prob': 0.025, 'lr_mult': 10.0, 'patience': 128, 'batch_norm': False, 'initialization': 'kaiming_normal'}
Loading model with 8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN from runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN/last.ckpt
Model: NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
    (12

<Figure size 3000x1800 with 1 Axes>

### Deep Lift


In [23]:
df_lift = get_attributions(spot_tuner, fun_control, attr_method="DeepLift",n_rel=10)
print(df_lift)
plot_attributions(df_lift,  attr_method="DeepLift")

GPU available: True (mps), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs



  | Name   | Type       | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------
0 | layers | Sequential | 169    | train | [2048, 10] | [2048, 1]
-----------------------------------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


train_model result: {'val_loss': 2867.108154296875, 'hp_metric': 2867.108154296875}
config: {'l1': 8, 'epochs': 2048, 'batch_size': 2048, 'act_fn': Swish(), 'optimizer': 'Adamax', 'dropout_prob': 0.025, 'lr_mult': 10.0, 'patience': 128, 'batch_norm': False, 'initialization': 'kaiming_normal'}
Loading model with 8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN from runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN/last.ckpt
Model: NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
   

<Figure size 3000x1800 with 1 Axes>

### Feature Ablation


In [24]:
df_fl = get_attributions(spot_tuner, fun_control, attr_method="FeatureAblation",n_rel=10)

GPU available: True (mps), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs



  | Name   | Type       | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------
0 | layers | Sequential | 169    | train | [2048, 10] | [2048, 1]
-----------------------------------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


`Trainer.fit` stopped: `max_epochs=2048` reached.


train_model result: {'val_loss': 19317.923828125, 'hp_metric': 19317.923828125}
config: {'l1': 8, 'epochs': 2048, 'batch_size': 2048, 'act_fn': Swish(), 'optimizer': 'Adamax', 'dropout_prob': 0.025, 'lr_mult': 10.0, 'patience': 128, 'batch_norm': False, 'initialization': 'kaiming_normal'}
Loading model with 8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN from runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN/last.ckpt
Model: NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
    (12

In [25]:
print(df_fl)
plot_attributions(df_fl, attr_method="FeatureAblation")

   Feature Index Feature  FeatureAblationAttribution
0              6  s3_hdl                    0.679725
1              2     bmi                    0.669086
2              9  s6_glu                    0.648128
3              4   s1_tc                   -0.451220
4              5  s2_ldl                   -0.425339
5              3      bp                    0.284479
6              0     age                    0.195145
7              7  s4_tch                   -0.185990
8              8  s5_ltg                   -0.088764
9              1     sex                    0.015983


<Figure size 3000x1800 with 1 Axes>

## Conductance


In [26]:
from spotpython.plot.xai import plot_conductance_last_layer, get_weights_conductance_last_layer
weights_last, layer_conductance_last = get_weights_conductance_last_layer(spot_tuner, fun_control)
plot_conductance_last_layer(weights_last, layer_conductance_last, figsize=(6, 6))

GPU available: True (mps), used: True


TPU available: False, using: 0 TPU cores


HPU available: False, using: 0 HPUs



  | Name   | Type       | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------
0 | layers | Sequential | 169    | train | [2048, 10] | [2048, 1]
-----------------------------------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


GPU available: True (mps), used: True


...

HPU available: False, using: 0 HPUs



  | Name   | Type       | Params | Mode  | In sizes   | Out sizes
-----------------------------------------------------------------------
0 | layers | Sequential | 169    | train | [2048, 10] | [2048, 1]
-----------------------------------------------------------------------
169       Trainable params
0         Non-trainable params
169       Total params
0.001     Total estimated model params size (MB)
15        Modules in train mode
0         Modules in eval mode


train_model result: {'val_loss': 4836.5693359375, 'hp_metric': 4836.5693359375}
config: {'l1': 8, 'epochs': 2048, 'batch_size': 2048, 'act_fn': Swish(), 'optimizer': 'Adamax', 'dropout_prob': 0.025, 'lr_mult': 10.0, 'patience': 128, 'batch_norm': False, 'initialization': 'kaiming_normal'}
Loading model with 8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN from runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN/last.ckpt
Model: NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
    (12

train_model result: {'val_loss': 3858.173095703125, 'hp_metric': 3858.173095703125}
config: {'l1': 8, 'epochs': 2048, 'batch_size': 2048, 'act_fn': Swish(), 'optimizer': 'Adamax', 'dropout_prob': 0.025, 'lr_mult': 10.0, 'patience': 128, 'batch_norm': False, 'initialization': 'kaiming_normal'}
Loading model with 8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN from runs/saved_models/8_2048_2048_Swish_Adamax_0.025_10.0_128_False_kaiming_normal_TRAIN/last.ckpt
Model: NNLinearRegressor(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=8, bias=True)
    (1): Swish()
    (2): Dropout(p=0.025, inplace=False)
    (3): Linear(in_features=8, out_features=4, bias=True)
    (4): Swish()
    (5): Dropout(p=0.025, inplace=False)
    (6): Linear(in_features=4, out_features=4, bias=True)
    (7): Swish()
    (8): Dropout(p=0.025, inplace=False)
    (9): Linear(in_features=4, out_features=2, bias=True)
    (10): Swish()
    (11): Dropout(p=0.025, inplace=False)
   

<Figure size 1800x1800 with 1 Axes>